In [17]:
from neo4j import GraphDatabase

# Connexion à la base de données Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "Gabardiop"

driver = GraphDatabase.driver(uri, auth=(username, password))

# Fonction pour créer un individu (nœud) dans Neo4j avec un nom complet en évitant les doublons
def creer_individu(tx, nom_complet):
    query = """
    MERGE (p:Person {nom_complet: $nom_complet})
    RETURN id(p) AS id
    """
    result = tx.run(query, nom_complet=nom_complet)
    return result.single()["id"]

# Fonction pour créer une relation parent-enfant entre deux nœuds en évitant les doublons de relation
def creer_relation(tx, parent_id, enfant_id):
    query = """
    MATCH (p:Person), (e:Person)
    WHERE id(p) = $parent_id AND id(e) = $enfant_id
    MERGE (p)-[:PARENT_DE]->(e)
    """
    tx.run(query, parent_id=parent_id, enfant_id=enfant_id)

# Fonction pour insérer un individu et créer des relations parent-enfant en évitant les doublons
def ajouter_individu_et_relations(tx, parent_nom_complet, enfants):
    parent_id = creer_individu(tx, parent_nom_complet)
    
    for enfant_nom_complet in enfants:
        enfant_id = creer_individu(tx, enfant_nom_complet)
        creer_relation(tx, parent_id, enfant_id)


# Fermer la connexion Neo4j
driver.close()


In [18]:
def ajouter_individu_et_relations(tx, nom_complet, relations, base_label):
    # Création ou mise à jour de l'individu avec un label de base
    tx.run(
        """
        MERGE (personne:Person:%s {nom_complet: $nom_complet})
        """ % base_label,
        nom_complet=nom_complet
    )
    
    # Ajout des relations pour chaque personne
    for relation in relations:
        tx.run(
            """
            MERGE (parent:Person:%s {nom_complet: $relation})
            MERGE (personne:Person:%s {nom_complet: $nom_complet})
            MERGE (personne)-[:PARENT_DE]->(parent)
            """ % (base_label, base_label),
            nom_complet=nom_complet,
            relation=relation
        )

# Mise à jour des relations pour le groupe 2 avec le label "Base2"
def ajouter_nouvelles_relations_groupe_2():
    with driver.session() as session:
        # Groupe 2: Base2
        session.write_transaction(ajouter_individu_et_relations, "Seune Ndiaye", [
            "Tame"
        ],  "Base2")
        # Tame (mère de Birame Khoudia Tame Ndiaye)
        session.write_transaction(ajouter_individu_et_relations, "Tame", [
            "Birame Khoudia Tame Ndiaye"
        ],  "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Souleymane Ndiaye", [
            "Maréme Ndiaye", "Astou Ndiaye", "Moustapha Ndiaye", "Oulimata Ndiaye",
            "Ismaila Ndiaye", "Ndeye Rosalie Ndiaye", "Madiagne Ndiaye", "Ibrahima Ndiaye",
            "Fatou Ndiaye", "Bassirou Ndiaye", "Seynabou Ndiaye", "Seune Ndiaye (Souleymane)"
        ], "Base2")

        session.write_transaction(ajouter_individu_et_relations, "Sakoki Ndiaye", ["Souleymane Ndiaye"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Birame Rouba Ndiaye", ["Sakoki Ndiaye", "Mamour Ndiaye"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Yirim Mbagnick Ndiaye", ["Birame Rouba Ndiaye"], "Base2")
        # session.write_transaction(ajouter_individu_et_relations, "Mamour Ndiaye", ["Aymérou Fama Ndiaye"], "Base2")
        # session.write_transaction(ajouter_individu_et_relations, "Biram Khoudia Tame Ndiaye", ["Sokhna Ndiaye"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Sokhna Ndiaye", ["Arame Bonko"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Arame Bonko", ["Safiétou Niang"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Safiétou Niang", [
            "Serigne Abdou Aziz Sy", "Serigne Mansour Sy", "Abib Sy", 
            "Sokhna Oumou Khalry Malick", "Sokhna Rokhaya Malick", "Sokhna Nafissatou Malick"
        ], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Kombarou Ndaw", ["Ngouye Ndiaye", "Fama Sy", "Seune Ndiaye"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Ngouye Ndiaye", ["Fatimata Ngouye"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Fatimata Ngouye", ["Seynabou"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Amadou Yélé", ["Seynabou", "Sakhéwar"], "Base2")
        session.write_transaction(ajouter_individu_et_relations, "Sakhéwar", ["Lat-Dior"], "Base2")

# Appeler la fonction pour ajouter les nouvelles relations du groupe 2
ajouter_nouvelles_relations_groupe_2()

# Fermer la connexion Neo4j
driver.close()


/tmp/ipykernel_55263/1560384858.py:24: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_55263/1560384858.py:26: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Seune Ndiaye", [
/tmp/ipykernel_55263/1560384858.py:30: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Tame", [
/tmp/ipykernel_55263/1560384858.py:33: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Souleymane Ndiaye", [
/tmp/ipykernel_55263/1560384858.py:39: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Sakoki Ndiaye", ["Souleymane Ndiaye"], "Base2")
/tmp/ipykernel_5526

In [19]:

# Mise à jour des relations sans créer de doublons
def ajouter_nouvelles_relations():
    with driver.session() as session:
        # Tame (mère de Birame Khoudia Tame Ndiaye)
        session.write_transaction(ajouter_individu_et_relations, "Birame Khoudia Tame Ndiaye", [
            "Sokhna Ndiaye"
        ],  "Base2")

# Appeler la fonction pour ajouter les nouvelles relations
ajouter_nouvelles_relations()

# Fermer la connexion Neo4j
driver.close()

/tmp/ipykernel_55263/2147057585.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_55263/2147057585.py:5: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Birame Khoudia Tame Ndiaye", [


In [20]:

# Mise à jour des relations sans créer de doublons
def ajouter_nouvelles_relations():
    with driver.session() as session:
        # Tame (mère de Birame Khoudia Tame Ndiaye)
        session.write_transaction(ajouter_individu_et_relations, "Birame Khoudia Tame Ndiaye", [
            "Sokhna Ndiaye"
        ],  "Base2")

# Appeler la fonction pour ajouter les nouvelles relations
ajouter_nouvelles_relations()

# Fermer la connexion Neo4j
driver.close()

/tmp/ipykernel_55263/2147057585.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_55263/2147057585.py:5: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Birame Khoudia Tame Ndiaye", [


In [21]:

# Mise à jour des relations sans créer de doublons
def ajouter_nouvelles_relations():
    with driver.session() as session:
        session.write_transaction(ajouter_individu_et_relations, "Fama Sy", [
            "Aymérou Fama Ndiaye",
            "Birame Rouba Ndiaye"
        ],  "Base2") 

# Appeler la fonction pour ajouter les nouvelles relations
ajouter_nouvelles_relations()

# Fermer la connexion Neo4j
driver.close()

/tmp/ipykernel_55263/636140617.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_55263/636140617.py:4: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Fama Sy", [


In [22]:

# # Mise à jour des relations sans créer de doublons
# def ajouter_nouvelles_relations():
#     with driver.session() as session:
#         session.write_transaction(ajouter_individu_et_relations, "Coumba Diouf", [
#             "Abdoulaye",
#             "Alboury Ndiaye",
#             "Macodou Ndiaye",
#             "Bouna Ndiaye",
#             "Laita Ndiaye",
#             "Sidy Ndiaye",
#             "Massamba Ndiaye",
#             "Cheikh Sadibou",
#             "Mass Ndiaye",
#             "Baty Ndiaye",
#             "Bigué Ndiaye",
#             "Fatou Ndiaye "
#         ]) 
        

# # Appeler la fonction pour ajouter les nouvelles relations
# ajouter_nouvelles_relations()

# # Fermer la connexion Neo4j
# driver.close()

In [23]:
# Mise à jour des relations sans créer de doublons
def ajouter_nouvelles_relations():
    with driver.session() as session:
        session.write_transaction(ajouter_individu_et_relations, "Birame Samba", [
            "Mandiaye Birame",
        ],  "Base2") 
        session.write_transaction(ajouter_individu_et_relations, "Mandiaye Birame", [
            "Mamour  Ndiaye",
        ],  "Base2") 
        session.write_transaction(ajouter_individu_et_relations, "Mamour  Ndiaye", [
            "Birame Rouba Mamour",
        ],  "Base2") 
        session.write_transaction(ajouter_individu_et_relations, "Birame Rouba Mamour", [
            "Mandiaye Khedj",
        ],  "Base2") 
        session.write_transaction(ajouter_individu_et_relations, "Mandiaye Khedj", [
            "Yirim Mbagnick Ndiaye",
        ],  "Base2") 

# Appeler la fonction pour ajouter les nouvelles relations
ajouter_nouvelles_relations()

# Fermer la connexion Neo4j
driver.close()

/tmp/ipykernel_55263/219261109.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_55263/219261109.py:4: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Birame Samba", [
/tmp/ipykernel_55263/219261109.py:7: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Mandiaye Birame", [
/tmp/ipykernel_55263/219261109.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Mamour  Ndiaye", [
/tmp/ipykernel_55263/219261109.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Birame Rouba Mamour", [
/tmp/ipykernel_55263/219261109.py:16: Depre

In [24]:
from neo4j import GraphDatabase

# Connexion à la base de données Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "Gabardiop"))

def ajouter_individu_et_relations(tx, nom_complet, relations, base_label):
    # Création ou mise à jour de l'individu avec un label de base
    tx.run(
        """
        MERGE (personne:Person:%s {nom_complet: $nom_complet})
        """ % base_label,
        nom_complet=nom_complet
    )
    
    # Ajout des relations pour chaque personne
    for relation in relations:
        # Vérifier si la relation existe déjà dans la base de données
        tx.run(
            """
            MERGE (relation:Person {nom_complet: $relation})
            MERGE (personne:Person:%s {nom_complet: $nom_complet})
            MERGE (personne)-[:PARENT_DE]->(relation)
            """ % base_label,
            nom_complet=nom_complet,
            relation=relation
        )

# Fonction pour ajouter de nouvelles relations sans créer de doublons
def ajouter_nouvelles_relations():
    with driver.session() as session:
        # Groupe 1: Base1
        session.write_transaction(ajouter_individu_et_relations, "Asta Madièye Diop", [
            "Baba Ndiaré Ndiaye"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Baba Ndiaré Ndiaye", [
            "Marie Baba Ndiaré Ndiaye",
            "Awa Baba Ndiaré Ndiaye",
            "Madiop Baba Ndiaré Ndiaye"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Marie Baba Ndiaré Ndiaye", [
            "Anta Sarr Ndiawar"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Anta Sarr Ndiawar", [
            "Oulimata Diouf"  # Assurez-vous que c'est l'Oulimata correcte
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Oulimata Diouf", [
            "Mar Seck", "Massène Diallo", "Alpha Ndiaye", "Iba Diagne", "Isse Tew Diagne", "Diarra Ouli"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Awa Baba Ndiaré Ndiaye", [
            "Penda Camara", "Djibril Camara"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Penda Camara", [
            "Ainina Diop", "Binta Yama Diop", "Magatte Diop Ibnou", "Babacar Diop", "Yaye Awa Diop", "Abdoulaye Kébé", "Souleymane kébé", "Amy Kébé"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Ibnou Diop Wara", [
            "Ainina Diop", "Binta Yama Diop", "Magatte Diop Ibnou", "Yaye Awa Diop","Babacar Diop"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Amadou Lamine Kébé", [
            "Abdoulaye Kébé", "Souleymane kébé", "Amy Kébé"
        ], "Base1")
        
        session.write_transaction(ajouter_individu_et_relations, "Djibril Camara", [
            "Malick Camara", "Latyr Camara", "Ibou Camara", "Mame Aida Camara"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Marguerite Boye", [
            "Malick Camara", "Latyr Camara", "Ibou Camara", "Mame Aida Camara"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Malick Camara", [
            "Margot", "Cheikh Demba Camara", 
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Adja Fat Ly", [
            "Margot", "Cheikh Demba Camara"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Mame Aida Camara", [
            "Binta Guèye"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Madiop Baba Ndiaré Ndiaye", [
            "Boly Ndiaye"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Boly Ndiaye", [
            "Awa Boly Ndiaye"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Awa Boly Ndiaye", [
            "Rokhaya Ndiaye Awa Boly"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Rokhaya Ndiaye Awa Boly", [
            "Serigne Babacar Sy"
        ], "Base1")

        session.write_transaction(ajouter_individu_et_relations, "Latyr Camara", [
            "Mame Marième Camara", "Margot Camara", "Yacine Camara"
        ], "Base1")

      
# Appeler la fonction pour ajouter les nouvelles relations
ajouter_nouvelles_relations()

# Fermer la connexion Neo4j
driver.close()


/tmp/ipykernel_55263/742818135.py:32: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Asta Madièye Diop", [
/tmp/ipykernel_55263/742818135.py:36: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Baba Ndiaré Ndiaye", [
/tmp/ipykernel_55263/742818135.py:42: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Marie Baba Ndiaré Ndiaye", [
/tmp/ipykernel_55263/742818135.py:46: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Anta Sarr Ndiawar", [
/tmp/ipykernel_55263/742818135.py:50: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Oulimata Diouf", [
/tmp/ipykernel_55263/742818135.py:5

In [25]:
# Fonction pour ajouter de nouvelles relations sans créer de doublons
def ajouter_nouvelles_relations():
    with driver.session() as session:
        # Groupe 3: Base3
        #famille WIlliam Diouf et son frère Edmond
     
        
        session.write_transaction(ajouter_individu_et_relations, "William Diouf", [
            "Oulimata  Diouf", "Rosalie Diouf"
        ], "Base3")
        session.write_transaction(ajouter_individu_et_relations, "Oulimata  Diouf", [
            "Mar Seck", "Massène Diallo", "Alpha Ndiaye", "Iba Diagne", "Isse Tew Diagne", "Diarra Ouli"
        ], "Base3")
        session.write_transaction(ajouter_individu_et_relations, "Rosalie Diouf", [
            "Souleymane Ndiaye","Amadou Moustapha Diagne","Moctar Diagne","Baba Diagne","Fatou Diagne","Astou Diagne","Amadou Abdoulaye Diagne"
        ], "Base3")        
        session.write_transaction(ajouter_individu_et_relations, "Djiby Diagne", [
            "Amadou Moustapha Diagne","Moctar Diagne","Baba Diagne","Fatou Diagne","Astou Diagne","Amadou Abdoulaye Diagne"
        ], "Base3")   
        session.write_transaction(ajouter_individu_et_relations, "Sakoki Ndiaye",
                                  ["Souleymane Ndiaye"], "Base3")
        session.write_transaction(ajouter_individu_et_relations, "Souleymane Ndiaye",
                                  ["Marème Ndiaye"], "Base3")
        session.write_transaction(ajouter_individu_et_relations, "Edmond Diouf",
                                  ["Mame Guitté Diouf"], "Base3")
        session.write_transaction(ajouter_individu_et_relations, "Mame Guitté Diouf",
                                  ["Astou Fall","Mame Pourmera Fall","Ousseynou Fall","Assane Fall","Ablaye Fall","Cheikh Fall"], "Base3")
        session.write_transaction(ajouter_individu_et_relations, "Astou Fall",
                                  ["Ndeye Tabara Diop","Ndeye Binta Diop","Pape Diop"], "Base3")
        session.write_transaction(ajouter_individu_et_relations, "Mame Pourmera Fall",
                                  ["Lamine Tew","Soppé Diop","Mame S. Diop"], "Base3")
        # session.write_transaction(ajouter_individu_et_relations, "Mar Seck",
        #                           ["Binta Yama Seck","Nafissatou Seck","Amineta Seck","Mouhamadou Moustapha Seck","Pathé Seck","Penda Ndiaye Seck","Mame Aida Seck","Marème Seck","Fatou Seck","Moustapha Seck","Boubacar Seck","Aissatou Seck","Magatte Seck","Warxa Seck"], "Base3")
        # session.write_transaction(ajouter_individu_et_relations, "Marième Ndiaye",
        #                           ["Binta Yama Seck","Nafissatou Seck","Aminta Seck","Mouhamadou Moustapha Seck","Pathé Seck","Penda Ndiaye Seck","Mame Aida Seck"], "Base3")
        
        
        

        #Famille Fatou Diallo Binta Sarr

        session.write_transaction(ajouter_individu_et_relations, "Binta Sarr",
                                  ["Fatou Diallo"], "Base4")
        session.write_transaction(ajouter_individu_et_relations, "Fatou Diallo",
                                  ["Marième Ndiaye","Aissatou Ndiaye", "Amadou Moustapha Ndiaye", "Oulimata Ndiaye","Rosalie Ndiaye","Madiagne Ndiaye","Babacar Dieng","Baaye Dieng","Madicke Dieng"], "Base4")
        
        session.write_transaction(ajouter_individu_et_relations, "Souleymane Ndiaye",
                                  ["Marième Ndiaye","Aissatou Ndiaye", "Amadou Moustapha Ndiaye", "Oulimata Ndiaye","Rosalie Ndiaye","Madiagne Ndiaye"], "Base4")
        
        session.write_transaction(ajouter_individu_et_relations, "Pape Thiaka Dieng",
                                  ["Babacar Dieng","Baaye Dieng","Madicke Dieng"], "Base4")
        
        
        #Famille Mar Seck et ses enfants
        session.write_transaction(ajouter_individu_et_relations, "Ndiogou Seck",
                                  ["Yatma Seck"], "Base5")
        session.write_transaction(ajouter_individu_et_relations, "Yatma Seck",
                                  ["Mar Seck","Malick Seck","Pathé Ndegg Seck","Ouleymatou Seck","Fary Seck"], "Base5")
        session.write_transaction(ajouter_individu_et_relations, "Marième Ndiaye",
                                  ["Binta Yama Seck","Nafissatou Seck","Aminta Seck","Mouhamadou Moustapha Seck","Pathé Seck","Penda Ndiaye Seck","Mame Aida Seck"], "Base5")
        
        session.write_transaction(ajouter_individu_et_relations, "Maguette Fall Dagana",
                                  ["Warxa Seck"], "Base5")
        
        session.write_transaction(ajouter_individu_et_relations, "Courra Dièye Sarr",
                                  ["Magatte Seck"], "Base5")
        
        session.write_transaction(ajouter_individu_et_relations, "Nafissatou Diop Attoumane",
                                  ["Marème Seck","Fatou Seck","Moustapha Seck","Boubacar Seck","Aissatou Seck"], "Base5")
        
        session.write_transaction(ajouter_individu_et_relations, "Mar Seck",
                                  ["Binta Yama Seck","Nafissatou Seck","Aminta Seck","Mouhamadou Moustapha Seck","Pathé Seck","Penda Ndiaye Seck","Mame Aida Seck","Marème Seck","Fatou Seck","Moustapha Seck","Boubacar Seck","Aissatou Seck","Magatte Seck","Warxa Seck"], "Base5")


        session.write_transaction(ajouter_individu_et_relations, "Marième Ndiaye",
                                  ["Binta Yama Seck","Nafissatou Seck","Aminta Seck","Mouhamadou Moustapha Seck","Pathé Seck","Penda Ndiaye Seck","Mame Aida Seck"], "Base6")
        
        session.write_transaction(ajouter_individu_et_relations, "Mar Seck",
                                  ["Binta Yama Seck","Nafissatou Seck","Aminta Seck","Mouhamadou Moustapha Seck","Pathé Seck","Penda Ndiaye Seck","Mame Aida Seck"], "Base6")
        
        session.write_transaction(ajouter_individu_et_relations, "Binta Yama Seck",
                                  ["Ndeye Wouly Diagne","Pape Mar Diagne","Idrissa Diagne","Tidjane Diagne","Madjiguène Diagne","Makhtar Diagne","Moussa Diagne","Diarra Diagne","Yatma Diagne","Nafissatou Diagne","Mame Aida Diagne"], "Base6")
        session.write_transaction(ajouter_individu_et_relations,"Nafissatou Seck",
                                  ["Magatte Diop","Mame Aissatou Diedhiou","Magatte Fall Diedhiou","Fatou Néné Diedhiou","Alphousseynou Diedhiou","Assane Diedhiou","Ismaila Diedhiou","MOuhameth Diedhiou"], "Base6") 
        session.write_transaction(ajouter_individu_et_relations,"Aminta Seck",
                                  ["Binta Sarr Thoub","Abdou Yacine Cissé","Safiétou Cissé","Ndèye Marième Cissé"], "Base6") 
        
        session.write_transaction(ajouter_individu_et_relations,"Mar Seck",
                                  ["Mouhamadou Moustapha Seck"], "Base7") 
        
        session.write_transaction(ajouter_individu_et_relations,"Marième Ndiaye",
                                  ["Mouhamadou Moustapha Seck"], "Base7") 
        session.write_transaction(ajouter_individu_et_relations,"Mouhamadou Moustapha Seck",
                                  ["Pape Mar Seck","Ndèye Tacko Seck","Lalla Seck","Famara Seck","Ndeye Birame Seck","Cheikh Seck","El Djiby Seck","Mame Khady"], "Base7") 
        session.write_transaction(ajouter_individu_et_relations,"Fatou Nar Diagne",
                                  ["Pape Mar Seck","Ndèye Tacko Seck","Lalla Seck","Famara Seck","Ndeye Birame Seck","Cheikh Seck","El Djiby Seck","Mame Khady"], "Base7") 
        session.write_transaction(ajouter_individu_et_relations,"Mouhamadou Moustapha Seck",
                                  ["Marième Seck","Ndoumbé Seck","Pathé Seck","Fatou Seck","Ndeye Ami Seck","Boubacar Seck","Bassirou Seck"], "Base7") 
        
        session.write_transaction(ajouter_individu_et_relations,"Fatou Pouye",
                                  ["Marième Seck","Ndoumbé Seck","Pathé Seck","Fatou Seck","Ndeye Ami Seck","Boubacar Seck","Bassirou Seck"], "Base7") 
        


        session.write_transaction(ajouter_individu_et_relations,"Mar Seck",
                                  ["Pathé Seck"], "Base8") 
        
        session.write_transaction(ajouter_individu_et_relations,"Marième Ndiaye",
                                  ["Pathé Seck"], "Base8") 

        session.write_transaction(ajouter_individu_et_relations,"Pathé Seck",
                                  ["Makha Seck","Penda Ndiaye Seck","Sokhna Oumou Kalsoume Seck","Pauline Seck","Aby Seck","Marième Aicha Seck","Marième Nora Seck"], "Base8") 
        

        
        session.write_transaction(ajouter_individu_et_relations,"Yacine Diop",
                                  ["Makha Seck","Penda Ndiaye Seck"], "Base8") 
        
        session.write_transaction(ajouter_individu_et_relations,"Ndèye Coumba Seck",
                                  ["Sokhna Oumou Kalsoume Seck","Pauline Seck","Aby Seck","Marième Aicha Seck"], "Base8")
        session.write_transaction(ajouter_individu_et_relations,"Laurence Ba",
                                  ["Marième Nora Seck"], "Base8") 
        
        #Descendance Fa Gaye et Nar Gaye
        session.write_transaction(ajouter_individu_et_relations, "Ndiogou Seck",
                                  ["Yatma  Seck"], "Base9")
        session.write_transaction(ajouter_individu_et_relations, "Fa Gaye",
                                  ["Mota  Sarr","Anna Sarr","Massar Fa Gaye","Balla Sarr","Demba Lo Sarr","Mandaw Sarr","Sokhna Fall","Médoune Sène","Awa Dièye"], "Base9")
        
        session.write_transaction(ajouter_individu_et_relations, "Sokhna Fall",
                                  ["Fatou Fall"], "Base9")
        session.write_transaction(ajouter_individu_et_relations, "Fatou Fall",
                                  ["Yatma Dièye"], "Base9")
        session.write_transaction(ajouter_individu_et_relations, "Yatma Dièye",
                                  [" Sokhna Mbacké Dièye","Moustapha Dièye", "Colé Yate Dièye", "Nafi Dièye","Aida Bayo","Rama Dièye","Idrissa Dièye","Doudou Diop Dièye"], "Base9")
        
        session.write_transaction(ajouter_individu_et_relations, "Sokhna Bayo Sarr",
                                  [" Sokhna Mbacké Dièye","Moustapha Dièye", "Colé Yate Dièye", "Nafi Dièye","Aida Bayo","Rama Dièye","Idrissa Dièye","Doudou Diop Dièye"], "Base9")
        session.write_transaction(ajouter_individu_et_relations, "Massar Fa Gaye",
                                  ["Penda Ndiaye Sarr"], "Base9")
        
        session.write_transaction(ajouter_individu_et_relations, "Gandj Dièye",
                                  ["Awa Dièye"], "Base9")
        session.write_transaction(ajouter_individu_et_relations, "Awa Dièye",
                                  ["Yacine Dièye Sall","Daly Sall"], "Base9")
        
        session.write_transaction(ajouter_individu_et_relations, "Mota  Sarr",
                                  ["Yatma  Seck"], "Base9")
        
        session.write_transaction(ajouter_individu_et_relations, "Yatma  Seck",
                                  ["Mar  Seck","Malick Seck","Pathé Ndegg Seck","Ouleymatou Seck","Fary Seck"], "Base9")
        
        session.write_transaction(ajouter_individu_et_relations, "Nar Gaye",
                                  ["Dame Awa Fall","Moussa Fall","Alioune Fall"], "Base9")
        
        #Descendance Mbous Mademba Seck

        session.write_transaction(ajouter_individu_et_relations, "Mbous Mademba",
                                  ["Masseck Fanta","Cheikh Malamine Seck","Fatoumata Seck","Mame Arama Ngandji Seck","Astou Seck"], "Base10")



# Appeler la fonction pour ajouter les nouvelles relations
ajouter_nouvelles_relations()

# Fermer la connexion Neo4j
driver.close()


/tmp/ipykernel_55263/625716158.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_55263/625716158.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "William Diouf", [
/tmp/ipykernel_55263/625716158.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Oulimata  Diouf", [
/tmp/ipykernel_55263/625716158.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Rosalie Diouf", [
/tmp/ipykernel_55263/625716158.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(ajouter_individu_et_relations, "Djiby Diagne", [
/tmp/ipykernel_55263/625716158.py:20: Deprecation

In [26]:
import graphviz
from graphviz import Digraph
from neo4j import GraphDatabase
from pdfrw import PdfReader, PdfWriter, PageMerge
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

# Connexion à Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Gabardiop"))

def fetch_tree_data(tx):
    query = """
    MATCH (p:Person)-[r:PARENT_DE]->(c:Person)
    RETURN p.nom_complet AS parent, type(r) AS relationship, c.nom_complet AS child
    """
    result = tx.run(query)
    return [(record["parent"], record["relationship"], record["child"]) for record in result]

# Générer l'arbre généalogique
def create_family_tree(tree_data):
    dot = Digraph()

    # Définir des styles globaux pour le graphique
    dot.attr(dpi='300', rankdir='TB', bgcolor='lightgray', nodesep='1', ranksep='1', size='10,10')

    # Définir des styles pour les nœuds
    dot.attr('node', shape='box', style='filled', fillcolor='lightblue', fontname='Arial', fontsize='14', color='black')

    # Définir des styles pour les arêtes
    dot.attr('edge', color='black', fontname='Arial', fontsize='12', style='dashed')

    family_dict = {}
    for parent, relationship, child in tree_data:
        if parent not in family_dict:
            family_dict[parent] = []
        family_dict[parent].append(child)

    # Ajouter les nœuds et les arêtes au graphique avec des styles
    for parent, children in family_dict.items():
        dot.node(str(parent), label=parent)
        for child in children:
            dot.node(str(child), label=child)
            dot.edge(str(parent), str(child))

    return dot

# Connexion à Neo4j et récupération des données
with driver.session() as session:
    tree_data = session.read_transaction(fetch_tree_data)

# Création et exportation de l'arbre généalogique
family_tree = create_family_tree(tree_data)
output_path = '/home/gblack98/Téléchargements/Gabar_family_tree'
family_tree.render(filename=output_path, cleanup=True)  # Génère family_tree.pdf dans Téléchargements

# Créer un fichier PDF temporaire avec la signature et les numéros de page
def create_signature_pdf(output_path):
    c = canvas.Canvas(output_path, pagesize=letter)
    width, height = letter
    
    c.setFont("Helvetica", 10)
    c.drawString(72, height - 50, "Créé par Nafissatou Seck")
    
    c.save()

# Ajouter la signature et les numéros de page au PDF existant
def add_signature_to_pdf(input_pdf, signature_pdf, output_pdf):
    original_pdf = PdfReader(input_pdf)
    signature = PdfReader(signature_pdf)
    writer = PdfWriter()

    # Assurer que signature.pdf a la même taille que le PDF principal
    for i, page in enumerate(original_pdf.pages):
        # Créer une page temporaire pour fusionner la signature
        c = canvas.Canvas("/tmp/temp_page.pdf", pagesize=letter)
        width, height = letter
        
        # Ajouter la signature à la page
        c.drawString(72, height - 50, "Créé par Nafissatou Seck")
        c.drawString(width - 72, height - 72, f"Page {i + 1}")
        c.save()
        
        # Ajouter la page temporaire et la page originale
        temp_page = PdfReader("/tmp/temp_page.pdf").pages[0]
        page = PageMerge(page).add(temp_page).render()
        writer.addpage(page)

    writer.write(output_pdf)

# Chemins des fichiers
signature_pdf_path = '/home/gblack98/Téléchargements/signature.pdf'
final_output_pdf_path = '/home/gblack98/Téléchargements/Gabar_family_tree_signed_nouveau.pdf'

# Générer le fichier PDF de signature
create_signature_pdf(signature_pdf_path)

# Ajouter la signature et les numéros de page
add_signature_to_pdf('/home/gblack98/Téléchargements/Gabar_family_tree.pdf', signature_pdf_path, final_output_pdf_path)


/tmp/ipykernel_55263/841756707.py:50: DeprecationWarning: read_transaction has been renamed to execute_read
  tree_data = session.read_transaction(fetch_tree_data)
[ERROR] tokens.py:226 Expected PDF /name object (line=12, col=16, token='% 1')


In [27]:
import graphviz
from graphviz import Digraph
from neo4j import GraphDatabase
from pdfrw import PdfReader, PdfWriter, PageMerge
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

# Connexion à Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Gabardiop"))

def fetch_tree_data(tx):
    query = """
    MATCH (p:Person)-[r:PARENT_DE]->(c:Person)
    RETURN p.nom_complet AS parent, type(r) AS relationship, c.nom_complet AS child
    """
    result = tx.run(query)
    return [(record["parent"], record["relationship"], record["child"]) for record in result]

# Générer l'arbre généalogique
def create_family_tree(tree_data):
    dot = Digraph()

    # Définir des styles globaux pour le graphique
    dot.attr(dpi='300', rankdir='TB', bgcolor='lightgray', nodesep='1', ranksep='1', size='10,10')

    # Définir des styles pour les nœuds
    dot.attr('node', shape='box', style='filled', fillcolor='lightblue', fontname='Arial', fontsize='14', color='black')

    # Définir des styles pour les arêtes
    dot.attr('edge', color='black', fontname='Arial', fontsize='12', style='dashed')

    family_dict = {}
    for parent, relationship, child in tree_data:
        if parent not in family_dict:
            family_dict[parent] = []
        family_dict[parent].append(child)

    # Ajouter les nœuds et les arêtes au graphique avec des styles
    for parent, children in family_dict.items():
        dot.node(str(parent), label=parent)
        for child in children:
            dot.node(str(child), label=child)
            dot.edge(str(parent), str(child))

    return dot

# Connexion à Neo4j et récupération des données
with driver.session() as session:
    tree_data = session.read_transaction(fetch_tree_data)

# Création et exportation de l'arbre généalogique
family_tree = create_family_tree(tree_data)
output_path = '/home/gblack98/Téléchargements/Gabar_family_tree'
family_tree.render(filename=output_path, cleanup=True)  # Génère family_tree.pdf dans Téléchargements

# Créer un fichier PDF temporaire avec la signature et les numéros de page
def create_signature_pdf(output_path):
    c = canvas.Canvas(output_path, pagesize=letter)
    width, height = letter
    
    # Ajustement de la position de la signature en bas à gauche
    c.setFont("Helvetica", 10)
    c.drawString(72, 30, "Créé par Nafissatou Seck")  # Ajustement de la position (72, 30 pour la signature en bas à gauche)
    
    c.save()

# Ajouter la signature et les numéros de page au PDF existant
def add_signature_to_pdf(input_pdf, signature_pdf, output_pdf):
    original_pdf = PdfReader(input_pdf)
    signature = PdfReader(signature_pdf)
    writer = PdfWriter()

    # Assurer que signature.pdf a la même taille que le PDF principal
    for i, page in enumerate(original_pdf.pages):
        # Créer une page temporaire pour fusionner la signature
        c = canvas.Canvas("/tmp/temp_page.pdf", pagesize=letter)
        width, height = letter
        
        # Ajouter la signature en bas à gauche
        c.drawString(72, 30, "Créé par Nafissatou Seck")  # Signature en bas à gauche
        
        # Ajouter le numéro de page en bas à droite
        c.drawString(width - 100, 30, f"Page {i + 1}")  # Ajustement pour le numéro de page en bas à droite
        
        c.save()
        
        # Ajouter la page temporaire et la page originale
        temp_page = PdfReader("/tmp/temp_page.pdf").pages[0]
        page = PageMerge(page).add(temp_page).render()
        writer.addpage(page)

    writer.write(output_pdf)

# Chemins des fichiers
signature_pdf_path = '/home/gblack98/Téléchargements/signature.pdf'
final_output_pdf_path = '/home/gblack98/Téléchargements/Gabar_family_tree_signed_correctedd.pdf'

# Générer le fichier PDF de signature
create_signature_pdf(signature_pdf_path)

# Ajouter la signature et les numéros de page
add_signature_to_pdf('/home/gblack98/Téléchargements/Gabar_family_tree.pdf', signature_pdf_path, final_output_pdf_path)


/tmp/ipykernel_55263/2771645763.py:50: DeprecationWarning: read_transaction has been renamed to execute_read
  tree_data = session.read_transaction(fetch_tree_data)
[ERROR] tokens.py:226 Expected PDF /name object (line=12, col=16, token='% 1')


In [28]:
import graphviz
from graphviz import Digraph
from neo4j import GraphDatabase

# Connexion à Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Gabardiop"))

def fetch_tree_data(tx):
    query = """
    MATCH (p:Person)-[r:PARENT_DE]->(c:Person)
    RETURN p.nom_complet AS parent, type(r) AS relationship, c.nom_complet AS child
    """
    result = tx.run(query)
    return [(record["parent"], record["relationship"], record["child"]) for record in result]

# Générer l'arbre généalogique avec des paramètres adaptés à une grande image
def create_family_tree(tree_data):
    dot = Digraph()

    # Définir des styles globaux pour le graphique
    dot.attr(dpi='300', rankdir='TB', bgcolor='lightgray', nodesep='2', ranksep='2', size='15,15')  # Taille augmentée, DPI à 300

    # Définir des styles pour les nœuds (taille et police augmentées)
    dot.attr('node', shape='box', style='filled', fillcolor='lightblue', fontname='Arial', fontsize='24', width='2', height='2')

    # Définir des styles pour les arêtes
    dot.attr('edge', color='black', fontname='Arial', fontsize='20', style='dashed')

    family_dict = {}
    for parent, relationship, child in tree_data:
        if parent not in family_dict:
            family_dict[parent] = []
        family_dict[parent].append(child)

    # Ajouter les nœuds et les arêtes au graphique
    for parent, children in family_dict.items():
        dot.node(str(parent), label=parent)
        for child in children:
            dot.node(str(child), label=child)
            dot.edge(str(parent), str(child))

    return dot

# Connexion à Neo4j et récupération des données
with driver.session() as session:
    tree_data = session.read_transaction(fetch_tree_data)

# Création de l'arbre généalogique
family_tree = create_family_tree(tree_data)

# Exporter l'arbre généalogique au format PNG avec une grande taille
output_image_path = '/home/gblack98/Téléchargements/Gabar_family_tree_image'
family_tree.render(filename=output_image_path, format='png', cleanup=True)


/tmp/ipykernel_55263/626611483.py:47: DeprecationWarning: read_transaction has been renamed to execute_read
  tree_data = session.read_transaction(fetch_tree_data)


'/home/gblack98/Téléchargements/Gabar_family_tree_image.png'

In [29]:
import graphviz
from graphviz import Digraph
from neo4j import GraphDatabase

# Connexion à Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Gabardiop"))

def fetch_tree_data(tx):
    query = """
    MATCH (p:Person)-[r:PARENT_DE]->(c:Person)
    RETURN p.nom_complet AS parent, type(r) AS relationship, c.nom_complet AS child
    """
    result = tx.run(query)
    return [(record["parent"], record["relationship"], record["child"]) for record in result]

# Générer l'arbre généalogique avec des paramètres adaptés à une grande image
def create_family_tree(tree_data):
    dot = Digraph()

    # Fond blanc, et taille de l'image plus grande avec une résolution plus élevée
    dot.attr(dpi='600', rankdir='TB', bgcolor='white', nodesep='2', ranksep='2', size='20,20')  # Fond blanc

    # Styles pour les nœuds : texte noir, fond clair pour bien voir les lignes noires
    dot.attr('node', shape='box', style='filled', fillcolor='lightyellow', fontname='Arial', fontsize='36', width='3', height='3', color='black')

    # Styles pour les arêtes : lignes noires bien visibles
    dot.attr('edge', color='black', fontname='Arial', fontsize='28', style='solid')  # Lignes solides noires

    family_dict = {}
    for parent, relationship, child in tree_data:
        if parent not in family_dict:
            family_dict[parent] = []
        family_dict[parent].append(child)

    # Ajouter les nœuds et les arêtes au graphique
    for parent, children in family_dict.items():
        dot.node(str(parent), label=parent)
        for child in children:
            dot.node(str(child), label=child)
            dot.edge(str(parent), str(child))

    return dot

# Connexion à Neo4j et récupération des données
with driver.session() as session:
    tree_data = session.read_transaction(fetch_tree_data)

# Création de l'arbre généalogique
family_tree = create_family_tree(tree_data)

# Exporter l'arbre généalogique au format PNG avec une grande taille
output_image_path = '/home/gblack98/Téléchargements/Gabar_family_tree_large_image'
family_tree.render(filename=output_image_path, format='png', cleanup=True)  # Générer une image PNG

# Optionnel : Générer en format vectoriel pour une impression plus précise (SVG)
# family_tree.render(filename=output_image_path, format='svg', cleanup=True)


/tmp/ipykernel_55263/3874447312.py:47: DeprecationWarning: read_transaction has been renamed to execute_read
  tree_data = session.read_transaction(fetch_tree_data)


'/home/gblack98/Téléchargements/Gabar_family_tree_large_image.png'

In [30]:
import graphviz
from graphviz import Digraph
from neo4j import GraphDatabase

# Connexion à Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Gabardiop"))

def fetch_tree_data(tx):
    query = """
    MATCH (p:Person)-[r:PARENT_DE]->(c:Person)
    RETURN p.nom_complet AS parent, type(r) AS relationship, c.nom_complet AS child
    """
    result = tx.run(query)
    return [(record["parent"], record["relationship"], record["child"]) for record in result]

# Générer l'arbre généalogique avec des paramètres adaptés à une grande image
def create_family_tree(tree_data):
    dot = Digraph()

    # Augmenter la taille de l'image et inverser l'orientation (rankdir='LR' pour "Left-to-Right")
    dot.attr(dpi='600', rankdir='LR', bgcolor='white', nodesep='2', ranksep='2', size='30,15')  # Augmenter la taille

    # Styles pour les nœuds : texte noir, fond plus contrasté (clair)
    dot.attr('node', shape='box', style='filled', fillcolor='lightblue', fontname='Arial', fontsize='36', width='3.5', height='2.5', color='black')

    # Styles pour les arêtes : lignes noires bien visibles
    dot.attr('edge', color='black', fontname='Arial', fontsize='28', style='solid')  # Lignes solides noires

    family_dict = {}
    for parent, relationship, child in tree_data:
        if parent not in family_dict:
            family_dict[parent] = []
        family_dict[parent].append(child)

    # Ajouter les nœuds et les arêtes au graphique
    for parent, children in family_dict.items():
        dot.node(str(parent), label=parent)
        for child in children:
            dot.node(str(child), label=child)
            dot.edge(str(parent), str(child))

    return dot

# Connexion à Neo4j et récupération des données
with driver.session() as session:
    tree_data = session.read_transaction(fetch_tree_data)

# Création de l'arbre généalogique
family_tree = create_family_tree(tree_data)

# Exporter l'arbre généalogique au format PNG avec une grande taille
output_image_path = '/home/gblack98/Téléchargements/gen/Gabar_family_tree_large_image'
family_tree.render(filename=output_image_path, format='png', cleanup=True)  # Générer une image PNG

# Optionnel : Générer en format vectoriel pour une impression plus précise (SVG)
family_tree.render(filename=output_image_path, format='svg', cleanup=True)  # SVG pour qualité d'impression maximale


/tmp/ipykernel_55263/4265031980.py:47: DeprecationWarning: read_transaction has been renamed to execute_read
  tree_data = session.read_transaction(fetch_tree_data)


'/home/gblack98/Téléchargements/gen/Gabar_family_tree_large_image.svg'

In [31]:
import graphviz
from graphviz import Digraph
from neo4j import GraphDatabase

# Connexion à Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Gabardiop"))

def fetch_tree_data(tx):
    query = """
    MATCH (p:Person)-[r:PARENT_DE]->(c:Person)
    RETURN p.nom_complet AS parent, type(r) AS relationship, c.nom_complet AS child
    """
    result = tx.run(query)
    return [(record["parent"], record["relationship"], record["child"]) for record in result]

# Générer l'arbre généalogique avec des paramètres adaptés à une grande image
def create_family_tree(tree_data):
    dot = Digraph()

    # Augmenter la taille de l'image, inversement horizontale (rankdir='LR') et fond contrasté (darkgray)
    dot.attr(dpi='600', rankdir='LR', bgcolor='darkgray', nodesep='2', ranksep='2', size='30,15')

    # Styles pour les nœuds : texte très grand, fond beige pour contraste avec le fond sombre
    dot.attr('node', shape='box', style='filled', fillcolor='beige', fontname='Arial', fontsize='48', width='4.5', height='3.5', color='black')

    # Styles pour les arêtes : lignes noires solides, texte d'une taille plus grande pour les relations
    dot.attr('edge', color='black', fontname='Arial', fontsize='36', style='solid')

    family_dict = {}
    for parent, relationship, child in tree_data:
        if parent not in family_dict:
            family_dict[parent] = []
        family_dict[parent].append(child)

    # Ajouter les nœuds et les arêtes au graphique
    for parent, children in family_dict.items():
        dot.node(str(parent), label=parent)
        for child in children:
            dot.node(str(child), label=child)
            dot.edge(str(parent), str(child))

    return dot

# Connexion à Neo4j et récupération des données
with driver.session() as session:
    tree_data = session.read_transaction(fetch_tree_data)

# Création de l'arbre généalogique
family_tree = create_family_tree(tree_data)

# Exporter l'arbre généalogique au format PNG avec une grande taille
output_image_path = '/home/gblack98/Téléchargements/gen/Gabar_family_tree_large_image'
family_tree.render(filename=output_image_path, format='png', cleanup=True)  # Générer une image PNG

# Optionnel : Générer en format vectoriel pour une impression plus précise (SVG)
family_tree.render(filename=output_image_path, format='svg', cleanup=True)  # SVG pour qualité d'impression maximale


/tmp/ipykernel_55263/53443609.py:47: DeprecationWarning: read_transaction has been renamed to execute_read
  tree_data = session.read_transaction(fetch_tree_data)


'/home/gblack98/Téléchargements/gen/Gabar_family_tree_large_image.svg'

In [32]:
# from neo4j import GraphDatabase

# # Connexion à la base de données Neo4j
# uri = "bolt://localhost:7687"
# username = "neo4j"
# password = "Gabardiop"

# driver = GraphDatabase.driver(uri, auth=(username, password))

# # Fonction pour créer un individu (nœud) dans Neo4j avec des labels de base
# def ajouter_individu_et_relations(tx, nom_complet, relations, base_labels):
#     # Création ou mise à jour de l'individu avec des labels de base
#     for base_label in base_labels:
#         tx.run(
#             """
#             MERGE (personne:Person:%s {nom_complet: $nom_complet})
#             """ % base_label,
#             nom_complet=nom_complet
#         )
    
#     # Ajout des relations pour chaque personne
#     for relation in relations:
#         for base_label in base_labels:
#             tx.run(
#                 """
#                 MERGE (parent:Person {nom_complet: $relation})
#                 MERGE (personne:Person:%s {nom_complet: $nom_complet})
#                 MERGE (personne)-[:PARENT_DE]->(parent)
#                 """ % base_label,
#                 nom_complet=nom_complet,
#                 relation=relation
#             )

# # Fonction pour ajouter les individus et leurs relations
# def ajouter_nouvelles_relations():
#     with driver.session() as session:
#         # Groupe 1: Base1
#         session.write_transaction(ajouter_individu_et_relations, "Asta Madièye Diop", [
#             "Baba Ndiaré Ndiaye"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Baba Ndiaré Ndiaye", [
#             "Marie Baba Ndiaré Ndiaye",
#             "Awa Baba Ndiaré Ndiaye",
#             "Madiop Baba Ndiaré Ndiaye"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Marie Baba Ndiaré Ndiaye", [
#             "Anta Sarr Ndiawar"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Anta Sarr Ndiawar", [
#             "Oulimata Diouf"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Oulimata Diouf", [
#             "Mar Seck", "Massène Diallo", "Alpha Ndiaye", "Iba Diagne", "Isse Tew Diagne", "Diarra Ouli"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Awa Baba Ndiaré Ndiaye", [
#             "Penda Camara", "Djibril Camara"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Penda Camara", [
#             "Ainina Diop", "Binta Yama Diop", "Magatte Diop Ibnou", "Babacar Diop", "Yaye Awa Diop", "Abdoulaye Kébé", "Souleymane kébé", "Amy Kébé"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Ibnou Diop Wara", [
#             "Ainina Diop", "Binta Yama Diop", "Magatte Diop Ibnou", "Yaye Awa Diop", "Babacar Diop"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Amadou Lamine Kébé", [
#             "Abdoulaye Kébé", "Souleymane kébé", "Amy Kébé"
#         ], ["Base1"])
        
#         session.write_transaction(ajouter_individu_et_relations, "Djibril Camara", [
#             "Malick Camara", "Latyr Camara", "Ibou Camara", "Mame Aida Camara"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Marguerite Boye", [
#             "Malick Camara", "Latyr Camara", "Ibou Camara", "Mame Aida Camara"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Malick Camara", [
#             "Margot", "Cheikh Demba Camara"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Adja Fat Ly", [
#             "Margot", "Cheikh Demba Camara"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Mame Aida Camara", [
#             "Binta Guèye"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Madiop Baba Ndiaré Ndiaye", [
#             "Boly Ndiaye"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Boly Ndiaye", [
#             "Awa Boly Ndiaye"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Awa Boly Ndiaye", [
#             "Rokhaya Ndiaye Awa Boly"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Rokhaya Ndiaye Awa Boly", [
#             "Serigne Babacar Sy"
#         ], ["Base1"])

#         session.write_transaction(ajouter_individu_et_relations, "Latyr Camara", [
#             "Mame Marième Camara", "Margot Camara", "Yacine Camara"
#         ], ["Base1"])

      

# # Fonction pour ajouter Oulimata Diouf dans les deux bases
# def ajouter_oulimata():
#     with driver.session() as session:
#         session.write_transaction(ajouter_individu_et_relations, "Oulimata Diouf", [], ["Base1", "Base3"])

# # Appeler les fonctions pour ajouter les nouvelles relations
# ajouter_nouvelles_relations()
# ajouter_oulimata()

# # Fermer la connexion Neo4j
# driver.close()
